# Welcome to spectral fitting with COSIpy classic

In this notebook, we'll perform a spectral fit on the Crab nebula and Centaurus A using simulated balloon flight data.

There are other simulated sources in the data set - Cygnus X-1, Vela, the 511 keV emission from positron annihilation and the Al-26 1.8 MeV decay line - but these are not explored in this notebook.

## Import packages.

We're using the COSIpy classic functions in COSIpy_dc1.py, response_dc1.py, and fit_dc1.py.

In [ ]:
from COSIpy_dc1 import *
import response_dc1 as response
from fit_dc1 import *

## Define file names.

DC1_combined_10x.tra.gz is a simulation of 4 point sources (Crab, Cen A, Cyg X-1, Vela), the 511 keV & Al26 lines, and Ling background.

CenA_BG_10x.tra.gz is a simulation of only Cen A and Ling background.

In [ ]:
data_dir = '../data_products' # directory containing data & response files
filename_combined = 'DC1_combined_10x.tra.gz' # combined simulation
filename_cenA = 'CenA_BG_10x.tra.gz' # Cen A simulation
response_filename = data_dir + '/Continuum_imaging_response.npz' # detector response
background_filename = data_dir + '/Scaled_Ling_BG_1x.npz' # background response
background_mode = 'from file'

## Define inputs.

You can perform the spectral fit on another point source, such as Cyg X-1 or Vela, by changing the latitude and longitude

In [ ]:
l_crab,b_crab = 184.55746, -5.78436 # Galactic longitude & latitude of Crab
l_cenA,b_cenA = 309.51584, 19.41709 # Galactic longitude & latitude of Cen A

ul = 3 # SNR limit for upper limits on spectral fit

## Read in simulation and define analysis object.

Read in the data set and create the main cosipy-classic “analysis_combined" object, which provides various functionalities to study the specified file. This cell usually takes a few minutes to run.

In [ ]:
analysis_combined = COSIpy(data_dir,filename_combined) # create analysis object
analysis_combined.read_COSI_DataSet() # read in data

# Bin the data
The data are binned into time, energy, ϕ and FISBEL. FISBEL is a unique index which specifies the χ and ψ dimensions of the CDS.

Calling "get_binned_data()" may take several minutes, depending on the size of the dataset and the number of bins. Keep an eye on memory here: if your time bins are very small, for example, this could be an expensive operation.

In [ ]:
#Define the bin sizes
Delta_T = 7200 # time bin size in seconds
energy_bin_edges = np.array([150,  220,  325,  480,  520,  765, 1120, 1650, 2350, 3450, 5000]) # energy bin edges in keV
pixel_size = 6. # pixel size in degrees

analysis_combined.dataset.time_binning_tags(time_bin_size=Delta_T) # time binning
analysis_combined.dataset.init_binning(energy_bin_edges=energy_bin_edges,pixel_size=pixel_size) # energy and pixel binning
analysis_combined.dataset.get_binned_data() # bin data

## Examine the shape of the binned data.

The binned data are contained in "analysis_combined.dataset.binned_data". This is a 4-dimensional object representing the 5 dimensions of the Compton data space (time, energy, ϕ, FISBEL).

This prints the shape of the binned data, the total time in the dataset, the number of time bins that have counts in them, and the number of counts in each time bin. Due to this energy range being so background dominated, the number of counts in each bin is very similar.

In [ ]:
print('Number of bins in each dimension (time, energy, ϕ, FISBEL):')
print(analysis_combined.dataset.binned_data.shape)
print()
print('Total time in dataset (s):')
print(analysis_combined.dataset.times.total_time)
print()
print('Number of populated time bins:')
print(analysis_combined.dataset.times.n_ph)
print()
print('Number of counts in each time bin: ')
print(analysis_combined.dataset.times.n_ph_t)

## Plot  raw spectrum & light curve.

In [ ]:
analysis_combined.dataset.plot_raw_spectrum()
plt.xscale('log')

analysis_combined.dataset.plot_lightcurve()

## Define the pointing object with the COSIpy pointing class.

The pointings refer to the direction/orientation of the telescope at each point in time. This cell usually takes a few minutes to run.

In [ ]:
pointing_combined = Pointing(dataset=analysis_combined.dataset) # definition of pointings (balloon stability + Earth rotation)

## Visualize the paths of the Crab & Cen A through the field-of-view.

This isn't necessary for the spectral fitting, but is illustrative for understanding the pointings and exposure of the point sources.

In [ ]:
plt.plot(pointing_combined.zpoins[:,0]+360,pointing_combined.zpoins[:,1],'o', label="COSI zenith pointing")
plt.plot(l_crab,b_crab,'*g',markersize=10, label="Crab")
plt.plot(l_cenA,b_cenA,'*r',markersize=10, label="Cen A")
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]')
plt.legend()

In [ ]:
analysis_combined.plot_elevation([l_crab],[b_crab],['Crab'])

analysis_combined.plot_elevation([l_cenA],[b_cenA],['Cen A'])

# Define the BG model.

In [ ]:
# Ling BG simulation to model atmospheric background
background_combined = BG(dataset=analysis_combined.dataset,mode=background_mode,filename=background_filename) # read in background

# Read in the Response Matrix

This usually takes a few minutes.

In [ ]:
# continuum response
rsp = response.SkyResponse(filename=response_filename,pixel_size=pixel_size) # read in detector response

## Explore the shape of the data space.

The shape of the response spans (Galactic latitude $b$, Galactic longitude $\ell$, Compton scattering angle $\phi$, FISBEL, energy). The shape of the data and background objects span (time, energy, Compton scattering angle, FISBEL), as explained above.

In [ ]:
print('Shape of response matrix (b, l, ϕ, FISBEL, energy):')
print(rsp.rsp.response_grid_normed_efinal.shape)
print()
print('Shape of binned data (time, energy, ϕ, FISBEL):')
print(analysis_combined.dataset.binned_data.shape)
print()
print('Shape of background model (time, energy, ϕ, FISBEL):')
print(np.shape(background_combined.bg_model))

# Calculate the point source response for the Crab.

In [ ]:
rsp.calculate_PS_response(analysis_combined.dataset,pointing_combined,l_crab,b_crab,1,background=background_combined,pixel_size=pixel_size,lookup=False)

## Plot light curves for the data, background & sky models.

This is plotted for the 220-325 keV energy bin. The sky model is normalized to 1.

In [ ]:
plt.plot(np.sum(analysis_combined.dataset.binned_data[:,1,:,:],axis=(1,2)), label="Data") # binned data light curve
plt.plot(np.sum(background_combined.bg_model_reduced[1],axis=1), label="Background model") # background model
plt.plot(np.sum(rsp.sky_response[1],axis=1)*1000, label="Sky model") # sky model
plt.xlabel('Time Bins')
plt.ylabel('Counts per Time Bin')
plt.legend()

# Extract the spectrum for the Crab.

For each energy bin individually, this determines the coefficients for the sky and background models that best match the data. It can take a few hours to run!

In [ ]:
result_crab = fit(analysis_combined.dataset,pointing_combined,rsp,background_combined) # create fitting object
result_crab.fit(iters=2000) # perform spectral fit using emcee (uses pointing definition, background model, & point source response)

## Plot the final count spectrum of the Crab.

Below is the spectrum (in counts/keV) of the Crab nebula!

The fitted value for the 480-520 keV bin is a bit low, which is likely due to the 511 keV line in the simulation.

The extracted spectrum data is saved as a .dat file.

In [ ]:
result_crab.plot_extracted_spectrum('crab_spectrum.dat')

# Analysis of Cen A

Now that we have successfully recovered the Crab spectrum from the full flight simulation, we will look at a slightly weaker source and discover some of the limitations with the current COSIpy classic implementation.

Since we’ve already loaded the COSI-balloon simulated data, response matrix and background model, we only need to redefine the point source sky model using Cen A’s coordinates and repeat the fit.

## Calculate the point source response for Cen A.

In [ ]:
rsp.calculate_PS_response(analysis_combined.dataset,pointing_combined,l_cenA,b_cenA,1,background=background_combined,pixel_size=pixel_size,lookup=False)

## Extract the spectrum for Cen A.

This can take a few hours to run!

In [ ]:
result_cenA = fit(analysis_combined.dataset,pointing_combined,rsp,background_combined) # create fitting object
result_cenA.fit(iters=2000) # perform spectral fit using emcee (uses pointing definition, background model, & point source response)

## Plot the spectrum of Cen A.

Below is the spectrum (in counts/keV) of the Cen A, with the extracted spectrum data saved as a .dat file.

The first plot includes error bars on each point, and the second shows upper limits where the signal-to-noise ratio (SNR) > 3.

The error bars for Cen A are very large, and almost all energy bins have upper limits. We believe this is due to the Crab's brightness interfering with the fit. To fix this, we need to include the Crab in the background model. Because the elevation of the Crab in COSI's field of view is changing over time, our background model would now be time-dependent. However, our current fitting algorithm only includes one background parameter, so we cannot handle a time-dependent background. Future versions of COSIpy will fix this issue.

In [ ]:
result_cenA.plot_extracted_spectrum('cenA_spectrum1.dat')
result_cenA.plot_extracted_spectrum('cenA_spectrum1.dat',ul=ul)

## Read in Cen A simulation and define analysis object.

To investigate why the fit is failing using the combined simulation, we will redo the Cen A analysis, but we will use a simulation that excludes the Crab and all other sources. The 'CenA_BG_10x.tra.gz' file includes only Cen A and the Ling background, and by repeating the above analysis, we will show that we can recover the Cen A count spectrum when the background model is static. For this analysis, we will only include the required cells. Please refer to the Crab analysis above for the details of each step.

This cell usually takes a few minutes to run.

In [ ]:
analysis_cenA = COSIpy(data_dir,filename_cenA) # create analysis object
analysis_cenA.read_COSI_DataSet() # read in data

## Bin the data.

In [ ]:
analysis_cenA.dataset.time_binning_tags(time_bin_size=Delta_T) # time binning
analysis_cenA.dataset.init_binning(energy_bin_edges=energy_bin_edges,pixel_size=pixel_size) # energy and pixel binning
analysis_cenA.dataset.get_binned_data() # bin data

## Define the pointing object.

This cell usually takes a few minutes to run

In [ ]:
pointing_cenA = Pointing(dataset=analysis_cenA.dataset)

## Define the background model.

In [ ]:
background_cenA = BG(dataset=analysis_cenA.dataset,mode=background_mode,filename=background_filename) # read in background

## Calculate the point source response for Cen A.

In [ ]:
rsp.calculate_PS_response(analysis_cenA.dataset,pointing_cenA,l_cenA,b_cenA,1,background=background_cenA,pixel_size=pixel_size,lookup=False)

## Plot light curves for data, background & sky models.

This is plotted for the 220-325 keV energy bin. The sky model is normalized to 1.

You can see that the data rates appear flatter here compared to the light curve above for the combined simulation. The Crab was mainly in COSI's field of view during the second half of the flight, and since we no longer have those additional counts, there is less variation in this light curve and sky model.

In [ ]:
plt.plot(np.sum(analysis_cenA.dataset.binned_data[:,1,:,:],axis=(1,2)), label="Data") # binned data light curve
plt.plot(np.sum(background_cenA.bg_model_reduced[1],axis=1), label="Background model") # background model
plt.plot(np.sum(rsp.sky_response[1],axis=1)*1000, label="Sky model") # sky model
plt.xlabel('Time Bins')
plt.ylabel('Counts per Time Bin')
plt.legend()

## Extract the spectrum for Cen A.

This can take a few hours to run!

In [ ]:
result_cenA2 = fit(analysis_cenA.dataset,pointing_cenA,rsp,background_cenA) # create fitting object
result_cenA2.fit(iters=2000) # perform spectral fit using emcee (uses pointing definition, background model, & point source response)

## Plot the final count spectrum of Cen A.

Below is the spectrum (in counts/keV) of Centaurus A!

When we use a simulation of only Cen A and background, we are able to recover the correct spectrum. This emphasizes the need for more sophisticated background models which include time variability and bright sources, and this will be a focus for the next Data Challenge.

In [ ]:
result_cenA2.plot_extracted_spectrum('cenA_only_spectrum1.dat')
result_cenA2.plot_extracted_spectrum('cenA_only_spectrum1.dat',ul=ul)